In [1]:
import pandas as pd 
import numpy as np

In [2]:
df1 = pd.read_csv('ex1.csv')
print(df1.shape)
df1.head()

(569, 32)


,Image ID,Diagnosis,Mean Radius,Mean Perimeter,Mean Area,Mean Texture,Mean Smoothness,Mean Compactness,Mean Concavity,Mean Concave Points,...,SE Radius,SE Perimeter,SE Area,SE Texture,SE Smoothness,SE Compactness,SE Concavity,SE Concave Points,SE Symmetry,SE Fractal Dim
0,842302,M,17.99,122.80,1001.0,10.38,0.12,0.27760,0.3001,0.1471,...,1.0950,8.589,153.40,0.9053,0.0064,0.0490,0.0537,0.0159,0.0300,0.0062
1,842517,M,20.57,132.90,1326.0,17.77,0.08,0.07864,0.0869,0.0702,...,0.5435,3.398,74.08,0.7339,0.0052,0.0131,0.0186,0.0134,0.0139,0.0035
2,84300903,M,19.69,130.00,1203.0,21.25,0.11,0.15990,0.1974,0.1279,...,0.7456,4.585,94.03,0.7869,0.0062,0.0401,0.0383,0.0206,0.0225,0.0046
3,84348301,M,11.42,77.58,386.1,20.38,0.14,0.28390,0.2414,0.1052,...,0.4956,3.445,27.23,1.1560,0.0091,0.0746,0.0566,0.0187,0.0596,0.0092
4,84358402,M,20.29,135.10,1297.0,14.34,0.10,0.13280,0.1980,0.1043,...,0.7572,5.438,94.44,0.7813,0.0115,0.0246,0.0569,0.0189,0.0176,0.0051


In [4]:
df1['Diagnosis'].value_counts()

B    357
M    212
Name: Diagnosis, dtype: int64

In [10]:
df1['Target'] = df1['Diagnosis'].replace({'B':0, 'M':1})
df1['Target'].value_counts()

0    357
1    212
Name: Target, dtype: int64

In [11]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import RobustScaler

In [14]:
pd.options.display.max_columns =100

In [17]:
X = df1.drop(columns=['Image ID','Diagnosis','Target'])
Y = df1['Target']

print(X.shape)
print(Y.shape)

(569, 30)
(569,)


In [19]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.3,
                                                   random_state=1234)
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

(398, 30)
(171, 30)
(398,)
(171,)


In [46]:
model = Pipeline( [('scaler',RobustScaler() ), 
                   ('modeling',DecisionTreeClassifier() )] )
model.fit(X_train, Y_train)

Pipeline(memory=None,
         steps=[('scaler',
                 RobustScaler(copy=True, quantile_range=(25.0, 75.0),
                              with_centering=True, with_scaling=True)),
                ('modeling',
                 DecisionTreeClassifier(class_weight=None, criterion='gini',
                                        max_depth=None, max_features=None,
                                        max_leaf_nodes=None,
                                        min_impurity_decrease=0.0,
                                        min_impurity_split=None,
                                        min_samples_leaf=1, min_samples_split=2,
                                        min_weight_fraction_leaf=0.0,
                                        presort=False, random_state=None,
                                        splitter='best'))],
         verbose=False)

In [47]:
param_list = {'modeling__max_depth':[3,4,5,6],
             'modeling__criterion':['gini','entropy'],
             'modeling__min_samples_split':[2,3,4,5],
             'modeling__class_weight':[None, 'balanced']}
model_gird = GridSearchCV(model, param_grid= param_list, cv =5)

In [48]:
model_gird.fit(X_train,Y_train)

GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('scaler',
                                        RobustScaler(copy=True,
                                                     quantile_range=(25.0,
                                                                     75.0),
                                                     with_centering=True,
                                                     with_scaling=True)),
                                       ('modeling',
                                        DecisionTreeClassifier(class_weight=None,
                                                               criterion='gini',
                                                               max_depth=None,
                                                               max_features=None,
                                                               max_leaf_nodes=None,
                              

In [49]:
best_model   = model_gird.best_estimator_
Y_pred_train = best_model.predict(X_train)
Y_pred_test  = best_model.predict(X_test)

In [50]:
from sklearn.metrics import accuracy_score ,f1_score

In [51]:
print('훈련데이터 셋 정확도', accuracy_score(Y_train, Y_pred_train))
print('테스트데이터 셋 정확도', accuracy_score(Y_test, Y_pred_test))
print('훈련데이터 셋 F1 - Score', f1_score(Y_train, Y_pred_train))
print('테스트데이터 셋 F1 - Score', f1_score(Y_test, Y_pred_test))

훈련데이터 셋 정확도 0.992462311557789
테스트데이터 셋 정확도 0.9181286549707602
훈련데이터 셋 F1 - Score 0.9897610921501707
테스트데이터 셋 F1 - Score 0.888888888888889


**RF**

In [31]:
model = Pipeline([('scaler',RobustScaler()), 
                  ('modeling',RandomForestClassifier()) ])
model.fit(X_train,Y_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Pipeline(memory=None,
         steps=[('scaler',
                 RobustScaler(copy=True, quantile_range=(25.0, 75.0),
                              with_centering=True, with_scaling=True)),
                ('modeling',
                 RandomForestClassifier(bootstrap=True, class_weight=None,
                                        criterion='gini', max_depth=None,
                                        max_features='auto',
                                        max_leaf_nodes=None,
                                        min_impurity_decrease=0.0,
                                        min_impurity_split=None,
                                        min_samples_leaf=1, min_samples_split=2,
                                        min_weight_fraction_leaf=0.0,
                                        n_estimators=10, n_jobs=None,
                                        oob_score=False, random_state=None,
                                        verbose=0, warm_start=False))],
        

In [35]:
param_list = {'modeling__max_depth':[3,4,5,6],
             'modeling__criterion':['gini','entropy'],
             'modeling__min_samples_split':[2,3,4,5],
             'modeling__class_weight':[None, 'balanced'],
             'modeling__n_estimators':[10,100,300,500]}
model_grid =GridSearchCV(model, param_grid=param_list, cv= 5, n_jobs= -1 )

In [36]:
model_grid.fit(X_train,Y_train)

GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('scaler',
                                        RobustScaler(copy=True,
                                                     quantile_range=(25.0,
                                                                     75.0),
                                                     with_centering=True,
                                                     with_scaling=True)),
                                       ('modeling',
                                        RandomForestClassifier(bootstrap=True,
                                                               class_weight=None,
                                                               criterion='gini',
                                                               max_depth=None,
                                                               max_features='auto',
                                 

In [39]:
best_model = model_grid.best_estimator_
Y_pred_train = best_model.predict(X_train)
Y_pred_test= best_model.predict(X_test)

In [40]:
print('훈련데이터 셋 정확도', accuracy_score(Y_train, Y_pred_train))
print('테스트데이터 셋 정확도', accuracy_score(Y_test, Y_pred_test))
print('훈련데이터 셋 F1 - Score', f1_score(Y_train, Y_pred_train))
print('테스트데이터 셋 F1 - Score', f1_score(Y_test, Y_pred_test))

훈련데이터 셋 정확도 0.9874371859296482
테스트데이터 셋 정확도 0.9239766081871345
훈련데이터 셋 F1 - Score 0.9825783972125435
테스트데이터 셋 F1 - Score 0.8943089430894309


* GradientBoosting Model에 대해서 학습 
* GirdSearchCV 활용 
* Accuracy / F1 score 로 평가 
* 0001jmp@gamil.com 송부 (다음주 수업 전) 